### Connects to API and pulls in data

In [208]:
### CONNECT TO API AND PULL IN DATA

import numpy as np
import pandas as pd
import json
from urllib2 import urlopen
from pprint import pprint
from IPython.display import display
import os.path #for accessing the json file
import sys
from datetime import datetime
reload(sys)
sys.setdefaultencoding('utf-8') #this is needed to allow special characters in urls and market names

# website = urlopen('https://predictit.org/api/marketdata/ticker/GORSUCH.SCOTUS.NEXTJUSTICE')
website = urlopen('https://predictit.org/api/marketdata/all')
predictit = website.read()


print os.path.isfile('test.json')
target = open('test.json','w')
target.truncate()
target.write(predictit)
target.close()

target = open('test.json','r')
with open('test.json') as data_file:
    predictit = json.load(data_file)
    
# pprint(predictit)

# print "predictit: "+str(type(predictit))
# print 'predictit["Markets"]): '+str(type(predictit["Markets"]))
# print 'predictit["Markets"][0]): '+str(type(predictit["Markets"][0]))
# print 'predictit["Markets"][0]["Contracts"]: '+str(type(predictit["Markets"][0]["Contracts"]))
# print 'predictit["Markets"][0]["Contracts"][0]: '+str(type(predictit["Markets"][0]["Contracts"][0]))

# contracts = 0
# for x in xrange(0,len(predictit["Markets"])):
#     contracts = contracts + len(predictit["Markets"][x]["Contracts"])

# print 'Number of markets: '+str(len(predictit["Markets"]))
# print 'Number of contracts: '+str(contracts)

### Process Data for  Database Upload

#### Step 1: Divide into tables

In [167]:
### PROCESS API DATA INTO TABLES ###

### Markets ###
markets_id = list()
markets_name = list()
markets_shortname = list()
markets_tickersymbol = list()
markets_url = list()
markets_timestamp = list()
for x in xrange(0,len(predictit["Markets"])):
    markets_id.append(predictit["Markets"][x]["ID"])
    markets_name.append(predictit["Markets"][x]["Name"])    
    markets_shortname.append(predictit["Markets"][x]["ShortName"])    
    markets_tickersymbol.append(predictit["Markets"][x]["TickerSymbol"])    
    markets_url.append(predictit["Markets"][x]["URL"])    
    markets_timestamp.append(predictit["Markets"][x]["TimeStamp"])    
    
markets = pd.DataFrame(data={"id":markets_id,"name":markets_name,"shortname":markets_shortname,"tickersymbol":markets_tickersymbol,"url":markets_url,"timestamp":markets_timestamp})
markets = markets.replace("N/A",np.nan)

### Contracts ###
contracts_id = list()
contracts_market_id = list()
contracts_longname = list()
contracts_name = list()
contracts_shortname = list()
contracts_date_end = list()
contracts_tickersymbol = list()
contracts_url = list()
contracts_timestamp = list()

for x in xrange(0,len(predictit["Markets"])):
    for y in xrange(0,len(predictit["Markets"][x]["Contracts"])):
        contracts_id.append(predictit["Markets"][x]["Contracts"][y]["ID"])
        contracts_market_id.append(predictit["Markets"][x]["ID"])
        contracts_longname.append(predictit["Markets"][x]["Contracts"][y]["LongName"])
        contracts_name.append(predictit["Markets"][x]["Contracts"][y]["Name"])
        contracts_shortname.append(predictit["Markets"][x]["Contracts"][y]["ShortName"])
        contracts_date_end.append(predictit["Markets"][x]["Contracts"][y]["DateEnd"])
        contracts_tickersymbol.append(predictit["Markets"][x]["Contracts"][y]["TickerSymbol"])
        contracts_url.append(predictit["Markets"][x]["Contracts"][y]["URL"])
        contracts_timestamp.append(predictit["Markets"][x]["TimeStamp"])

contracts = pd.DataFrame({"id":contracts_id,"market_id":contracts_market_id,"name":contracts_name,"longname":contracts_longname,"name":contracts_name,"shortname":contracts_shortname,"date_end":contracts_date_end,"tickersymbol":contracts_tickersymbol,"url":contracts_url,"timestamp":contracts_timestamp})
contracts = contracts.replace("N/A",np.nan)

### Prices ###
prices_contract_id = list()
prices_buy_no = list()
prices_buy_yes = list()
prices_sell_no = list()
prices_sell_yes = list()
prices_timestamp = list()

for x in xrange(0,len(predictit["Markets"])):
    for y in xrange(0,len(predictit["Markets"][x]["Contracts"])):
        prices_contract_id.append(predictit["Markets"][x]["Contracts"][y]["ID"])
        prices_buy_no.append(predictit["Markets"][x]["Contracts"][y]["BestBuyNoCost"])
        prices_buy_yes.append(predictit["Markets"][x]["Contracts"][y]["BestBuyYesCost"])
        prices_sell_no.append(predictit["Markets"][x]["Contracts"][y]["BestSellNoCost"])
        prices_sell_yes.append(predictit["Markets"][x]["Contracts"][y]["BestSellYesCost"])
        prices_timestamp.append(predictit["Markets"][x]["TimeStamp"])

prices = pd.DataFrame({"contract_id":prices_contract_id,"buy_no":prices_buy_no,"buy_yes":prices_buy_yes,"sell_no":prices_sell_no,"sell_yes":prices_sell_yes,"timestamp":contracts_timestamp})


# print "Markets Shape: "+str(markets.shape)
# print "Contracts Shape: "+str(contracts.shape)
# print "Prices Shape: "+str(prices.shape)


#### Step 2: Determine Required Market and Contract Updates

In [178]:
### IDENTIFY NEW/CLOSED MARKETS AND CONTRACTS ###

import psycopg2

try:
    conn = psycopg2.connect("dbname='postgres' user='jeremy' host='localhost' password=''")
except:
    print "I am unable to connect to the database"
    
cursor = conn.cursor()

### Markets ###
cursor.execute("""select id from markets where active = true""")

db_markets = cursor.fetchall()

active_markets = list()
for x in xrange(0,len(db_markets)):
    active_markets.append(db_markets[x][0])

new_markets = markets.loc[~markets['id'].isin(active_markets)]
new_markets["active"] = ["TRUE"]*len(new_markets)
new_markets.rename(columns={'timestamp':'created_at'}, inplace=True)

closed_markets = [x for x in active_markets if x not in list(markets["id"])]


### Contracts ###  
cursor.execute("""select id from contracts where active = true""")
db_contracts = cursor.fetchall()

active_contracts = list()
for x in xrange(0,len(db_contracts)):
    active_contracts.append(db_contracts[x][0])

new_contracts = contracts.loc[~contracts['id'].isin(active_contracts)]
new_contracts["active"] = ["TRUE"]*len(new_contracts)
new_contracts.rename(columns={'timestamp':'created_at'}, inplace=True)

closed_contracts = [x for x in active_contracts if x not in list(contracts["id"])]


# print contracts.shape
# print new_contracts.shape
# print closed_contracts

# print markets.shape
# print new_markets.shape
# print closed_markets


### Postgres Work

In [180]:
### MARKETS UPLOAD ###
for x in new_markets.index.values:
    cursor.execute("""
        insert into markets (
            id
            ,name
            ,shortname
            ,active
            ,created_at
            ,tickersymbol
            ,url
        )
        values (
            '"""+str(new_markets["id"][x])+"""'
            ,'"""+str(new_markets["name"][x]).replace("'","''")+"""'
            ,'"""+str(new_markets["shortname"][x]).replace("'","''")+"""'
            ,'"""+str(new_markets["active"][x])+"""'
            ,'"""+str(new_markets["created_at"][x])+"""'
            ,'"""+str(new_markets["tickersymbol"][x])+"""'
            ,'"""+str(new_markets["url"][x]).replace("'","''")+"""'
        )
        """)


In [181]:
### CONTRACTS UPLOAD ###
for x in new_contracts.index.values:
    if pd.isnull(new_contracts["date_end"][x]):
        skip = "--"
    else: 
        skip = ""
    
    
    cursor.execute("""
        insert into contracts (
            id
            ,market_id
            ,longname
            ,name
            ,shortname
            ,active
            """+str(skip)+""",date_end
            ,created_at
            ,tickersymbol
            ,url
        )
        values (
            '"""+str(new_contracts["id"][x])+"""'
            ,'"""+str(new_contracts["market_id"][x])+"""'
            ,'"""+str(new_contracts["longname"][x]).replace("'","''")+"""'
            ,'"""+str(new_contracts["name"][x]).replace("'","''")+"""'
            ,'"""+str(new_contracts["shortname"][x]).replace("'","''")+"""'
            ,'"""+str(new_contracts["active"][x])+"""'
            """+str(skip)+""",'"""+str(new_contracts["date_end"][x])+"""'
            ,'"""+str(new_contracts["created_at"][x])+"""'
            ,'"""+str(new_contracts["tickersymbol"][x])+"""'
            ,'"""+str(new_contracts["url"][x]).replace("'","''")+"""'
        )
        """)


In [182]:
### PRICES UPLOAD ###
for x in prices.index.values:
    
    if pd.isnull(prices["buy_no"][x]):
        skip_bn = "--"
    else: 
        skip_bn = ""
        
    if pd.isnull(prices["buy_yes"][x]):
        skip_by = "--"
    else: 
        skip_by = ""

    if pd.isnull(prices["sell_no"][x]):
        skip_sn = "--"
    else: 
        skip_sn = ""
        
    if pd.isnull(prices["sell_yes"][x]):
        skip_sy = "--"
    else: 
        skip_sy = ""
        
        
    cursor.execute("""
        insert into prices (
            contract_id
            ,timestamp
            """+str(skip_bn)+""",buy_no
            """+str(skip_by)+""",buy_yes
            """+str(skip_sn)+""",sell_no
            """+str(skip_sy)+""",sell_yes
        )
        values (
            '"""+str(prices["contract_id"][x])+"""'
            ,'"""+str(prices["timestamp"][x])+"""'
            """+str(skip_bn)+""",'"""+str(prices["buy_no"][x])+"""'
            """+str(skip_by)+""",'"""+str(prices["buy_yes"][x])+"""'
            """+str(skip_sn)+""",'"""+str(prices["sell_no"][x])+"""'
            """+str(skip_sy)+""",'"""+str(prices["sell_yes"][x])+"""'
        )
        """)


In [183]:
### CLOSE CONTRACTS ###
for x in closed_contracts:
    cursor.execute("""
        update contracts
        set active = false
        ,closed_at = TIMESTAMP WITH TIME ZONE '"""+(contracts["timestamp"][0])+"""'
        where id = """+str(x)+"""
        ;""")

In [184]:
### CLOSE MARKETS ###
for x in closed_markets:
    cursor.execute("""
        update markets
        set active = false
        ,closed_at = TIMESTAMP WITH TIME ZONE '"""+(markets["timestamp"][0])+"""'
        where id = """+str(x)+"""
        ;""")

In [185]:
### COMMIT CHANGES TO DATABASE ###
conn.commit()

In [219]:
### DISPLAY UPDATES MADE ###
display(str(datetime.now())+' update:')
display(str(len(prices))+' prices uploaded.')
display(str(len(list(new_markets["id"])))+' Markets opened: '+str(list(new_markets["id"])))
display(str(len(closed_markets))+' Markets closed: '+str(closed_markets))
display(str(len(list(new_contracts["id"])))+' Contracts opened: '+str(list(new_contracts["id"])))
display(str(len(closed_contracts))+' Contracts closed: '+str(closed_contracts))

'2017-02-13 12:03:56.684062 update:'

'763 prices uploaded.'

'152 Markets opened: [2198, 2404, 2405, 2406, 2449, 2454, 2455, 2461, 2463, 2468, 2493, 2634, 2635, 2636, 2654, 2662, 2663, 2664, 2666, 2667, 2668, 2672, 2685, 2698, 2699, 2703, 2704, 2708, 2709, 2710, 2721, 2723, 2725, 2726, 2733, 2734, 2742, 2743, 2744, 2745, 2747, 2748, 2749, 2750, 2751, 2753, 2754, 2755, 2760, 2761, 2762, 2766, 2767, 2768, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777, 2778, 2779, 2785, 2786, 2787, 2788, 2789, 2790, 2792, 2793, 2794, 2795, 2797, 2800, 2801, 2802, 2813, 2819, 2826, 2829, 2850, 2851, 2852, 2853, 2854, 2855, 2863, 2865, 2866, 2867, 2872, 2873, 2874, 2875, 2881, 2882, 2884, 2886, 2889, 2890, 2891, 2896, 2901, 2902, 2903, 2906, 2907, 2909, 2912, 2914, 2917, 2918, 2919, 2920, 2921, 2924, 2925, 2926, 2928, 2932, 2933, 2935, 2936, 2937, 2938, 2939, 2940, 2941, 2944, 2945, 2947, 2948, 2949, 2950, 2951, 2952, 2953, 2954, 2955, 2956, 2957, 2960, 2961, 2962, 2963, 2964, 2965, 2966, 2967, 2969]'

'0 Markets closed: []'

'763 Contracts opened: [4899, 2837, 2832, 2833, 2834, 2835, 2836, 2838, 2839, 2840, 2841, 2842, 2843, 2844, 4188, 4566, 4892, 4893, 4894, 4895, 4896, 4897, 4898, 4900, 4901, 3414, 3415, 3416, 3417, 3418, 3422, 5074, 3421, 3429, 3420, 3428, 5379, 3424, 3419, 3423, 3425, 3426, 3430, 3522, 3523, 4700, 3572, 3568, 3569, 5143, 3570, 3577, 3566, 3567, 3571, 3573, 3576, 3578, 3585, 3586, 3587, 3589, 3588, 4534, 3590, 3591, 3592, 3593, 4406, 4407, 4408, 4479, 4480, 4508, 4509, 4533, 4576, 4578, 4611, 4614, 4761, 3613, 3616, 3618, 4920, 3614, 3615, 3617, 3619, 3620, 3621, 3622, 3623, 3624, 3625, 3626, 3627, 4866, 4885, 3631, 3630, 3629, 3633, 3632, 3634, 3635, 3636, 3637, 3638, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3658, 3659, 3660, 3661, 3662, 3663, 3682, 4218, 4375, 4416, 4424, 4475, 4476, 4477, 4478, 4530, 4537, 4609, 4610, 4657, 4685, 5323, 4481, 3711, 3712, 3713, 3714, 3715, 3738, 4397, 4498, 4503, 4511, 4527, 5305, 4085, 4084, 4086, 4087, 5094, 4093, 4094, 4095, 4096, 4607, 4652, 4764

'0 Contracts closed: []'